In [554]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

## 1. movielens : 영화 평점 데이터를 활용하여 다음을 실시해보세요

1. 데이터 탐색 : NA 처리

2. 영화별 성별 평점 평균을 산출하여 성별로 상위/하위 10개 영화의 목록 추출 및 도식화 

3. 영화 평점 정보가 300건 이상 있는 영화에 대하여 여성에게 인기가 높은 상위 10개 영화

4. 남녀간 호불호가 큰 영화는?

5. 성별에 관계없이 호불호가 큰 영화는?(표준편차 활용)

In [220]:
movies = pd.read_fwf('data/movielens/movies.dat', header=None)
movies

,0,1,2
0,1::Toy Story (1995)::Animation|Children's|Comedy,NaN,NaN
1,2::Jumanji (1995)::Adventure|Children's|Fantasy,NaN,NaN
2,3::Grumpier Old Men (1995)::Comedy|Romance,NaN,NaN
3,4::Waiting to Exhale (1995)::Comedy|Drama,NaN,NaN
4,5::Father of the Bride Part II (1995)::Comedy,NaN,NaN
...,...,...,...
3878,3948::Meet the Parents (2000)::Comedy,NaN,NaN
3879,3949::Requiem for a Dream (2000)::Drama,NaN,NaN
3880,3950::Tigerland (2000)::Drama,NaN,NaN
3881,3951::Two Family House (2000)::Drama,NaN,NaN


In [181]:
ratings = pd.read_fwf('data/movielens/ratings.dat', header=None)
ratings

,0
0,1::1193::5::978300760
1,1::661::3::978302109
2,1::914::3::978301968
3,1::3408::4::978300275
4,1::2355::5::978824291
...,...
1000204,6040::1091::1::956716
1000205,6040::1094::5::956704
1000206,6040::562::5::9567047
1000207,6040::1096::4::956715


In [173]:
users = pd.read_fwf('data/movielens/users.dat', header=None)
users

,0
0,1::F::1::10::48067
1,2::M::56::16::70072
2,3::M::25::15::55117
3,4::M::45::7::02460
4,5::M::25::20::55455
...,...
6035,6036::F::25::15::3260
6036,6037::F::45::1::76006
6037,6038::F::56::1::14706
6038,6039::F::45::0::01060


---

데이터 탐색 : NA 처리

In [221]:
movies = movies.fillna(value="")
test = movies[0]
test[movies[1]!=""] = test[movies[1]!=""] + " " + movies[movies[1]!=""][1]
test[movies[2]!=""] = test[movies[2]!=""] + " " + movies[movies[2]!=""][2]
movies = test.str.split("::")
movies = list(movies)
movies = pd.DataFrame(movies, columns = ['movie id','title','genres'])

movies

,movie id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [182]:
ratings = ratings[0].str.split("::")
ratings = list(ratings)
ratings = pd.DataFrame(ratings, columns=['user id', 'movie id', 'rating', 'time stamp'])
ratings

,user id,movie id,rating,time stamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716
1000205,6040,1094,5,956704
1000206,6040,562,5,9567047
1000207,6040,1096,4,956715


In [183]:
users = users[0].str.split("::")
users = list(users)
users = pd.DataFrame(users, columns=['user id', 'gender', 'age', 'occupation', 'zip code'])
users

,user id,gender,age,occupation,zip code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,3260
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


In [185]:
ratings = ratings.fillna(value="")
users = users.fillna(value="")

In [264]:
ratings['rating'] = ratings['rating'].astype(int)
ratings['time stamp'] = ratings['time stamp'].astype(float)
users['age'] = users['age'].astype(int)
users['occupation'] = users['occupation'].astype(int)

---

영화별 성별 평점 평균을 산출하여 성별로 상위/하위 10개 영화의 목록 추출 및 도식화 

In [293]:
df1 = ratings[['user id', 'movie id', 'rating']]
df1 = pd.merge(df1, users[['user id', 'gender']], how="left")
r_means = df1.groupby(['movie id','gender'], as_index=False).mean()
r_means = r_means.sort_values(by='rating', ascending=False)
r_means = pd.merge(r_means, movies[['movie id', 'title']], how="left")
r_means = r_means[['movie id','title','gender','rating']]
r_means

,movie id,title,gender,rating
0,787,"Gate of Heavenly Peace, The (1995)",M,5.0
1,3233,Smashing Time (1967),M,5.0
2,3012,Battling Butler (1926),F,5.0
3,2444,24 7: Twenty Four Seven (1997),F,5.0
4,3382,Song of Freedom (1936),F,5.0
...,...,...,...,...
7147,843,Lotto Land (1995),M,1.0
7148,2256,Parasite (1982),F,1.0
7149,667,Bloodsport 2 (1995),F,1.0
7150,3899,Circus (2000),F,1.0


In [353]:
r_means_M = r_means[r_means['gender']=="M"]
r_means_M = r_means_M[['title','rating']] 
r_means_M.index = np.arange(1,len(r_means_M)+1,1)

r_means_F = r_means[r_means['gender']=="F"]
r_means_F = r_means_F[['title','rating']] 
r_means_F.index = np.arange(1,len(r_means_F)+1,1)

In [355]:
print("남자 인기영화 상위 10")
r_means_M.head(10)

남자 인기영화 상위 10


,title,rating
1,"Gate of Heavenly Peace, The (1995)",5.0
2,Smashing Time (1967),5.0
3,Ulysses (Ulisse) (1954),5.0
4,Small Wonders (1996),5.0
5,"Baby, The (1973)",5.0
6,Lured (1947),5.0
7,Follow the Bitch (1998),5.0
8,Dangerous Game (1993),5.0
9,"Bells, The (1926)",5.0
10,Angela (1995),5.0


In [356]:
print("남자 인기영화 하위 10")
r_means_M.tail(10)

남자 인기영화 하위 10


,title,rating
3662,Venice/Venice (1992),1.0
3663,Underworld (1997),1.0
3664,Theodore Rex (1995),1.0
3665,Kestrel's Eye (Falkens �ga) (1998),1.0
3666,Phat Beach (1996),1.0
3667,Shadows (Cienie) (1988),1.0
3668,Hillbillys in a Haunted House (1967),1.0
3669,Mutters Courage (1995),1.0
3670,Sleepover (1995),1.0
3671,Lotto Land (1995),1.0


In [357]:
print("여자 인기영화 상위 10")
r_means_F.head(10)

여자 인기영화 상위 10


,title,rating
1,Battling Butler (1926),5.0
2,24 7: Twenty Four Seven (1997),5.0
3,Song of Freedom (1936),5.0
4,Message to Love: The Isle of Wight Festival (1...,5.0
5,Raw Deal (1948),5.0
6,Twice Upon a Yesterday (1998),5.0
7,Ayn Rand: A Sense of Life (1997),5.0
8,Dancemaker (1998),5.0
9,"Big Combo, The (1955)",5.0
10,Clean Slate (Coup de Torchon) (1981),5.0


In [358]:
print("여자 인기영화 하위 10")
r_means_F.tail(10)

여자 인기영화 하위 10


,title,rating
3472,"Crow: Salvation, The (2000)",1.0
3473,"Alan Smithee Film: Burn Hollywood Burn, An (1997)",1.0
3474,Alien Escape (1995),1.0
3475,Diamonds (1999),1.0
3476,Spring Fever USA (a.k.a. Lauderdale) (1989),1.0
3477,Prom Night III: The Last Kiss (1989),1.0
3478,Parasite (1982),1.0
3479,Bloodsport 2 (1995),1.0
3480,Circus (2000),1.0
3481,In God's Hands (1998),1.0


---

영화 평점 정보가 300건 이상 있는 영화에 대하여 여성에게 인기가 높은 상위 10개 영화

In [318]:
r_count = df1.groupby(['movie id','gender'], as_index=False).count()
r_count.columns = ['movie id', 'gender', 'user id', 'count']
r_count

,movie id,gender,user id,count
0,1,F,591,591
1,1,M,1486,1486
2,10,F,134,134
3,10,M,754,754
4,100,F,21,21
...,...,...,...,...
7147,997,M,18,18
7148,998,F,20,20
7149,998,M,73,73
7150,999,F,45,45


In [319]:
r_count = pd.merge(r_count, r_means, how="left")
r_count = r_count[['movie id','title','gender','rating','count']]
r_count

,movie id,title,gender,rating,count
0,1,Toy Story (1995),F,4.187817,591
1,1,Toy Story (1995),M,4.130552,1486
2,10,GoldenEye (1995),F,3.470149,134
3,10,GoldenEye (1995),M,3.553050,754
4,100,City Hall (1996),F,2.571429,21
...,...,...,...,...,...
7147,997,Caught (1996),M,3.277778,18
7148,998,Set It Off (1996),F,3.450000,20
7149,998,Set It Off (1996),M,2.890411,73
7150,999,2 Days in the Valley (1996),F,3.488889,45


In [360]:
r_count300F = r_count.loc[(r_count['count'] >= 300) & (r_count['gender'] == "F")]
r_count300F = r_count300F.sort_values(by='rating', ascending=False)
r_count300F_top10 = r_count300F.head(10)
r_count300F_top10.index = np.arange(1,11,1)
r_count300F_top10[['title','rating']]

,title,rating
1,Schindler's List (1993),4.562602
2,"Shawshank Redemption, The (1994)",4.539075
3,To Kill a Mockingbird (1962),4.536667
4,"Usual Suspects, The (1995)",4.513317
5,"Sixth Sense, The (1999)",4.477410
6,Life Is Beautiful (La Vita � bella) (1997),4.422343
7,North by Northwest (1959),4.364458
8,"Wizard of Oz, The (1939)",4.355030
9,Amadeus (1984),4.346734
10,"Princess Bride, The (1987)",4.342767


---

남녀간 호불호가 큰 영화는?

성별에 관계없이 호불호가 큰 영화는?(표준편차 활용)

In [487]:
# 10건 이상의 리뷰가 있는 영화만 취급

r_count10 = r_count.loc[r_count['count'] >= 10]
r_count10

,movie id,title,gender,rating,count
0,1,Toy Story (1995),F,4.187817,591
1,1,Toy Story (1995),M,4.130552,1486
2,10,GoldenEye (1995),F,3.470149,134
3,10,GoldenEye (1995),M,3.553050,754
4,100,City Hall (1996),F,2.571429,21
...,...,...,...,...,...
7147,997,Caught (1996),M,3.277778,18
7148,998,Set It Off (1996),F,3.450000,20
7149,998,Set It Off (1996),M,2.890411,73
7150,999,2 Days in the Valley (1996),F,3.488889,45


In [517]:
r_gender = r_count10.copy()
r_F = r_gender[r_gender['gender']=="F"].copy()
r_F['rating'] = -r_F['rating']
r_M = r_gender[r_gender['gender']=="M"].copy()
r_gender = pd.concat([r_M,r_F])
r_gender = r_gender[['movie id', 'title', 'gender', 'rating']].groupby(['movie id', 'title'], as_index=False).sum()
r_gender['rating'] = r_gender['rating'].abs()
r_gender.columns = ['movie id','title','diff_gender']
r_gender = r_gender.sort_values(by='diff_gender', ascending=False)
r_gender = r_gender.reset_index(drop=True)
r_gender[['title','diff_gender']].head(10)

,title,diff_gender
0,Sanjuro (1962),4.639344
1,For All Mankind (1989),4.583333
2,"Great Day in Harlem, A (1994)",4.218750
3,Cold Fever (� k�ldum klaka) (1994),4.181818
4,Before the Rain (Pred dozhdot) (1994),4.173913
5,Lonely Are the Brave (1962),4.162791
6,"Last Days, The (1998)",4.150000
7,Chushingura (1962),4.111111
8,Joe the King (1999),4.090909
9,Vagabond (Sans toit ni loi) (1985),4.062500


In [408]:
r_std1 = df1[['movie id','rating']].groupby('movie id').std()
r_std1 = r_std1.sort_values(by='rating', ascending=False)
r_std1 = r_std1.dropna()
r_std1= r_std1.reset_index()
r_std1.columns = ['movie id', 'std']
r_std1 = pd.merge(r_std1,movies[['movie id','title']], how='left')

r_std1_top10 = r_std1[['title','std']].head(10)
r_std1_top10.index = np.arange(1,11,1)
r_std1_top10

,title,std
1,Foreign Student (1994),2.828427
2,Criminal Lovers (Les Amants Criminels) (1999),2.309401
3,Sunset Park (1996),2.121320
4,Better Living (1998),2.121320
5,"Neon Bible, The (1995)",2.121320
6,Zachariah (1971),2.121320
7,"Living Dead Girl, The (La Morte Vivante) (1982)",2.121320
8,Eaten Alive (1976),2.121320
9,Talk of Angels (1998),2.121320
10,Identification of a Woman (Identificazione di ...,2.121320


---

# 2. sample_1.xlsx 파일을 불러와서 다음을 수행하세요


- 1. 국적코드, 성별, 입국객수 3개 컬럼 사용

- 2. 데이터 정보 확인하기

- 3. 데이터 기초 통계량 확인

- 4-1. 입국객수 컬럼 추출

- 4-2. 국적코드, 입국객수 2개 컬럼 추출

- 4-3. 기준년월 컬럼 생성하여 2019-11로 모든 값 입력

- 5-1. sample_1에서 성별 남성인 사람 추출

- 5-2. sample_1에서 입국객수 15만명 이상 서브세팅

- 5-3. sample_1에서 남성이면서 입국객수 10만명 이상 서브세팅

- 5-4. sample_1에서 국적코드 A01과 A18 서브세팅

- 6. sample_codemaster.xlsx 불러와서 sample_1.xlsx 파일과 병합하기

- 7. sample_2.xlsx 불러와서 기준년월 2019-12 값 넣고 위 6번 데이터프레임 아래에 병합

- 8. 최종 데이터 엑셀파일로 내보내기


In [639]:
sam = pd.read_excel("data/sample_1.xlsx", header=None)
sam

,0,1,2,3
0,2019년 11월 입국객 수,NaN,NaN,NaN
1,국적코드,성별,입국객수,전년동기
2,A01,남성,106320,85815
3,A01,여성,191436,125241
4,A31,남성,319,299
5,A31,여성,42,54
6,A18,남성,158912,124486
7,A18,여성,232943,163466
8,총 합계,689972,NaN,NaN
9,전년동기,499361,NaN,NaN


In [640]:
sam1 = pd.DataFrame(sam.iloc[2:8])
sam1.columns=sam.iloc[1]
sam1

1,국적코드,성별,입국객수,전년동기
2,A01,남성,106320,85815
3,A01,여성,191436,125241
4,A31,남성,319,299
5,A31,여성,42,54
6,A18,남성,158912,124486
7,A18,여성,232943,163466


In [641]:
sam1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 2 to 7
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   국적코드    6 non-null      object
 1   성별      6 non-null      object
 2   입국객수    6 non-null      object
 3   전년동기    6 non-null      object
dtypes: object(4)
memory usage: 324.0+ bytes


In [642]:
sam1[['입국객수','전년동기']] = sam1[['입국객수','전년동기']].astype(int)
sam1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 2 to 7
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   국적코드    6 non-null      object
 1   성별      6 non-null      object
 2   입국객수    6 non-null      int32 
 3   전년동기    6 non-null      int32 
dtypes: int32(2), object(2)
memory usage: 276.0+ bytes


In [643]:
sam1.describe()

1,입국객수,전년동기
count,6.000000,6.000000
mean,114995.333333,83226.833333
std,98105.752006,68858.227184
min,42.000000,54.000000
25%,26819.250000,21678.000000
50%,132616.000000,105150.500000
75%,183305.000000,125052.250000
max,232943.000000,163466.000000


In [644]:
sam1['입국객수']

2    106320
3    191436
4       319
5        42
6    158912
7    232943
Name: 입국객수, dtype: int32

In [645]:
sam1[['국적코드','입국객수']]

1,국적코드,입국객수
2,A01,106320
3,A01,191436
4,A31,319
5,A31,42
6,A18,158912
7,A18,232943


In [646]:
sam1['기준년월'] = '2019-11'
sam1

1,국적코드,성별,입국객수,전년동기,기준년월
2,A01,남성,106320,85815,2019-11
3,A01,여성,191436,125241,2019-11
4,A31,남성,319,299,2019-11
5,A31,여성,42,54,2019-11
6,A18,남성,158912,124486,2019-11
7,A18,여성,232943,163466,2019-11


In [647]:
sam1[sam1['성별']=='남성']

1,국적코드,성별,입국객수,전년동기,기준년월
2,A01,남성,106320,85815,2019-11
4,A31,남성,319,299,2019-11
6,A18,남성,158912,124486,2019-11


In [648]:
sam1[sam1['입국객수']>=150000]

1,국적코드,성별,입국객수,전년동기,기준년월
3,A01,여성,191436,125241,2019-11
6,A18,남성,158912,124486,2019-11
7,A18,여성,232943,163466,2019-11


In [649]:
sam1[(sam1['성별']=='남성') & (sam1['입국객수']>=100000)]

1,국적코드,성별,입국객수,전년동기,기준년월
2,A01,남성,106320,85815,2019-11
6,A18,남성,158912,124486,2019-11


In [650]:
sam1[(sam1['국적코드'] == 'A01') | (sam1['국적코드'] == 'A18')]

1,국적코드,성별,입국객수,전년동기,기준년월
2,A01,남성,106320,85815,2019-11
3,A01,여성,191436,125241,2019-11
6,A18,남성,158912,124486,2019-11
7,A18,여성,232943,163466,2019-11


In [651]:
sam_cm = pd.read_excel("data/sample_codemaster.xlsx")
sam_cm

,국적코드,국적명
0,A01,일본
1,A02,대만
2,A03,홍콩
3,A18,중국
4,A19,이란
5,A22,우즈베키스탄
6,A23,카자흐스탄
7,A99,아시아 기타


In [652]:
sam1 = pd.merge(sam1, sam_cm, how='outer')
sam1

,국적코드,성별,입국객수,전년동기,기준년월,국적명
0,A01,남성,106320.0,85815.0,2019-11,일본
1,A01,여성,191436.0,125241.0,2019-11,일본
2,A31,남성,319.0,299.0,2019-11,NaN
3,A31,여성,42.0,54.0,2019-11,NaN
4,A18,남성,158912.0,124486.0,2019-11,중국
5,A18,여성,232943.0,163466.0,2019-11,중국
6,A02,NaN,NaN,NaN,NaN,대만
7,A03,NaN,NaN,NaN,NaN,홍콩
8,A19,NaN,NaN,NaN,NaN,이란
9,A22,NaN,NaN,NaN,NaN,우즈베키스탄


In [653]:
sam = pd.read_excel("data/sample_2.xlsx", header=None)
sam

,0,1,2,3
0,2019년 12월 입국객 수,NaN,NaN,NaN
1,국적코드,성별,입국객수,전년동기
2,A01,남성,92556,75992
3,A01,여성,163737,115397
4,A18,남성,155540,129908
5,A18,여성,249023,191236
6,총 합계,660856,NaN,NaN
7,전년동기,512533,NaN,NaN


In [654]:
sam2 = pd.DataFrame(sam.iloc[2:6])
sam2.columns=sam.iloc[1]
sam2['기준년월'] = '2019-12'
sam2

1,국적코드,성별,입국객수,전년동기,기준년월
2,A01,남성,92556,75992,2019-12
3,A01,여성,163737,115397,2019-12
4,A18,남성,155540,129908,2019-12
5,A18,여성,249023,191236,2019-12


In [655]:
sam = pd.concat([sam1,sam2])
sam

,국적코드,성별,입국객수,전년동기,기준년월,국적명
0,A01,남성,106320,85815,2019-11,일본
1,A01,여성,191436,125241,2019-11,일본
2,A31,남성,319,299,2019-11,NaN
3,A31,여성,42,54,2019-11,NaN
4,A18,남성,158912,124486,2019-11,중국
5,A18,여성,232943,163466,2019-11,중국
6,A02,NaN,NaN,NaN,NaN,대만
7,A03,NaN,NaN,NaN,NaN,홍콩
8,A19,NaN,NaN,NaN,NaN,이란
9,A22,NaN,NaN,NaN,NaN,우즈베키스탄


In [660]:
sam.to_excel("data/sample_result.xlsx")

---

# 3. 기상청 xml 데이터를 파싱하여 wf 데이터를 아래와 같이 데이터프레임으로 만들어보세요

- 주의 : 기상데이터의 tag를 처리하지 않고 진행하다 보면 wf 컬럼과 같이 데이터에 []가 포함되므로 이러한 경우 문자열형의 데이터를 가지는 wf2 컬럼을 생성해주세요

In [661]:
from bs4 import BeautifulSoup
import urllib.request as req
import os.path

In [662]:
url = 'http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=108' # 기상청 육상 중기예보
res = req.urlopen(url)

In [663]:
soup = BeautifulSoup(res,'html.parser')
soup

<?xml version="1.0" encoding="utf-8" ?>
<rss version="2.0">
<channel>
<title>기상청 육상 중기예보</title>
<link/>http://www.kma.go.kr/weather/forecast/mid-term_01.jsp
<description>기상청 날씨 웹서비스</description>
<language>ko</language>
<generator>기상청</generator>
<pubdate>2020년 09월 08일 (화)요일 06:00</pubdate>
<item>
<author>기상청</author>
<category>육상중기예보</category>
<title>전국 육상 중기예보 - 2020년 09월 08일 (화)요일 06:00 발표</title>
<link/>http://www.kma.go.kr/weather/forecast/mid-term_01.jsp
<guid>http://www.kma.go.kr/weather/forecast/mid-term_01.jsp</guid>
<description>
<header>
<title>전국 육상중기예보</title>
<tm>202009080600</tm>
<wf><![CDATA[○ (강수) 11일(금)부터 13일(일)까지 강원영동에 비가 오겠습니다.<br />○ (기온) 이번 예보기간의 낮 기온은 22~30도로 어제(20~27도)보다 높겠습니다.<br />○ (주말전망) 12일(토)과 13일(일)은 전국이 가끔 구름많겠으나, 강원영동은 흐리고 비가 오겠고, 아침 기온은 16~21도, 낮 기온은 22~30도 분포를 보이겠습니다.<br /><br />* 당분간 우리나라 상공에 차가운 공기가 머물면서 불안정이 강화되어 강수구역 및 시점의 변동성이 크겠으니, 앞으로 발표되는 기상정보를 참고하기 바랍니다.]]></wf>
</header>
<body>
<location wl_ver="3">
<province>서울ㆍ인천ㆍ경기도</province>
<city>서울<

In [666]:
a = soup.find_all('location')
len(a)

41

In [679]:
result_df = pd.DataFrame({'index':[], 'wf':[]})
result_df

,index,wf


In [680]:
for idx in range(len(a)):
    temp = pd.DataFrame({'index':a[idx].find('city').string,
                         'wf':a[idx].find_all('wf')})
    result_df = pd.concat([result_df,temp])
result_df

,index,wf
0,서울,[구름많음]
1,서울,[흐림]
2,서울,[흐림]
3,서울,[흐림]
4,서울,[맑음]
...,...,...
8,서귀포,[구름많음]
9,서귀포,[흐림]
10,서귀포,[구름많음]
11,서귀포,[흐림]


In [692]:
result_df2 = result_df.copy()

for idx in range(len(result_df)):
    result_df2['wf2'] = result_df2.iloc[idx,1].string

result_df2

,index,wf,wf2
0,서울,[구름많음],흐림
1,서울,[흐림],흐림
2,서울,[흐림],흐림
3,서울,[흐림],흐림
4,서울,[맑음],흐림
...,...,...,...
8,서귀포,[구름많음],흐림
9,서귀포,[흐림],흐림
10,서귀포,[구름많음],흐림
11,서귀포,[흐림],흐림


In [693]:
result_df2[['index','wf','wf2']].sort_values('index')

,index,wf,wf2
11,강릉,[흐림],흐림
12,강릉,[흐림],흐림
9,강릉,[흐림],흐림
8,강릉,[흐림],흐림
7,강릉,[구름많음],흐림
...,...,...,...
3,홍성,[구름많음],흐림
2,홍성,[구름많음],흐림
1,홍성,[구름많음],흐림
0,홍성,[구름많음],흐림
